In [66]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline

%matplotlib inline

## Về dataset này

Data Science Job Salaries dataset:
- `work_year`: The year the salary was paid.
- `experience_level`: The experience level in the job during the year
- `employment_type`: The type of employment for the role
- `job_title`: The role worked in during the year.
- `salary`: the total gross salary amount paid.
- `salary_currency`: The currency of the salary paid as an ISO 4217 currency code.
- `salaryinusd`: The salary in USD.
- `employee_residence`: Employee's primary country of residence in during the work year as an ISO 3166 country code.
- `remote_ratio`: The overall amount of work done remotely.
- `company_location`: The country of the employer's main office or contracting branch.
- `company_size`: The median number of people that worked for the company during the year.

## Khám phá dữ liệu

### Đọc dữ liệu từ file

In [67]:
# YOUR CODE HERE
# Đọc file
df = pd.read_csv('ds_salaries.csv')


# Drop các cột salary, salary_currency, Unnamed: 0
df.drop(df[['salary','salary_currency','Unnamed: 0']],axis=1, inplace=True)

In [68]:
# TEST
df.head(10)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,US,50,US,L
5,2020,EN,FT,Data Analyst,72000,US,100,US,L
6,2020,SE,FT,Lead Data Scientist,190000,US,100,US,S
7,2020,MI,FT,Data Scientist,35735,HU,50,HU,L
8,2020,MI,FT,Business Data Analyst,135000,US,100,US,L
9,2020,SE,FT,Lead Data Engineer,125000,NZ,50,NZ,S


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [69]:
# YOUR CODE HERE
shape = df.shape

In [70]:
# TEST
assert shape == (607, 9)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

### Dữ liệu có các dòng bị lặp không?

In [71]:
# YOUR CODE HERE
num_duplicated_rows = df.index.duplicated(keep='first').sum()

In [72]:
# TEST
assert num_duplicated_rows == 0

### Mỗi cột có ý nghĩa gì?

Ý nghĩa của các cột

- `work_year`: The year the salary was paid.
- `experience_level`: The experience level in the job during the year
- `employment_type`: The type of employment for the role
- `job_title`: The role worked in during the year.
- `salaryinusd`: The salary in USD.
- `employee_residence`: Employee's primary country of residence in during the work year as an ISO 3166 country code.
- `remote_ratio`: The overall amount of work done remotely.
- `company_location`: The country of the employer's main office or contracting branch.
- `company_size`: The median number of people that worked for the company during the year.

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [73]:
# YOUR CODE HERE
dtypes = df.dtypes

In [74]:
dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [75]:
# TEST
int_cols = set(dtypes[(dtypes==np.int32) | (dtypes==np.int64)].index)
assert int_cols == {'work_year', 'salary_in_usd', 'remote_ratio'}
object_cols = set(dtypes[dtypes == object].index)
assert object_cols == {'experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size'}

### Với mỗi cột có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [76]:
# YOUR CODE HERE
num_col_info_df = df.select_dtypes(exclude='object')
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

In [77]:
num_col_info_df = num_col_info_df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max])

In [78]:
# TEST
assert num_col_info_df.shape == (6, 3)
data = num_col_info_df.loc[['missing_ratio', 'min', 'lower_quartile', 'median', 'upper_quartile', 'max'],
                           ['work_year', 'salary_in_usd', 'remote_ratio']].values
correct_data = np.array([[        0.0,          0.0,        0,],
                         [     2020.0,       2859.0,       0.0],
                         [     2021.0,      62726.0,      50.0],
                         [     2022.0,     101570.0,     100.0],
                         [     2022.0,     150000.0,     100.0],
                         [     2022.0,     600000.0,     100.0]])

### Với mỗi cột có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [79]:
# YOUR CODE HERE
cat_col_info_df = df.select_dtypes(include='object')
cat_col_info_df.head()

,experience_level,employment_type,job_title,employee_residence,company_location,company_size
0,MI,FT,Data Scientist,DE,DE,L
1,SE,FT,Machine Learning Scientist,JP,JP,S
2,SE,FT,Big Data Engineer,GB,GB,M
3,MI,FT,Product Data Analyst,HN,HN,S
4,SE,FT,Machine Learning Engineer,US,US,L


In [80]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

In [81]:
cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df.shape

(3, 6)

In [82]:
cat_col_info_df

,experience_level,employment_type,job_title,employee_residence,company_location,company_size
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
num_values,4,4,50,57,50,3
value_ratios,"{'SE': 46.1, 'MI': 35.1, 'EN': 14.5, 'EX': 4.3}","{'FT': 96.9, 'PT': 1.6, 'CT': 0.8, 'FL': 0.7}","{'Data Scientist': 23.6, 'Data Engineer': 21.7...","{'US': 54.7, 'GB': 7.2, 'IN': 4.9, 'CA': 4.8, ...","{'US': 58.5, 'GB': 7.7, 'CA': 4.9, 'DE': 4.6, ...","{'M': 53.7, 'L': 32.6, 'S': 13.7}"


In [83]:
c = cat_col_info_df['company_location']
c.loc['value_ratios']['AT']

0.7

In [84]:
# TEST
c = cat_col_info_df['experience_level']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 4
assert c.loc['value_ratios']['SE'] == 46.1

c = cat_col_info_df['employment_type']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 4
assert c.loc['value_ratios']['PT'] == 1.6

c = cat_col_info_df['job_title']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 50
assert c.loc['value_ratios']['Machine Learning Manager'] == 0.2

c = cat_col_info_df['employee_residence']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 57
assert c.loc['value_ratios']['IT'] == 0.7

c = cat_col_info_df['company_location']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 50
assert c.loc['value_ratios']['AT'] == 0.7

c = cat_col_info_df['company_size']
assert c.loc['missing_ratio'] == 0.0
assert c.loc['num_values'] == 3
assert c.loc['value_ratios']['L'] == 32.6

In [85]:
df = df.replace({'Machine Learning Scientist': 'Data Scientist',
                 'Product Data Analyst': 'Data Analyst',
                 'Lead Data Scientist': 'Data Scientist',
                 'Business Data Analyst': 'Data Analyst',
                 'Lead Data Analyst': 'Data Analyst',
                 'Data Science Consultant': 'Data Scientist',
                 'BI Data Analyst': 'Data Analyst',
                 'Director of Data Science': 'Data Scientist',
                 'Research Scientist': 'Data Scientist',
                 'Machine Learning Manager': 'Data Scientist',
                 'AI Scientist': 'Data Scientist',
                 'Principal Data Scientist': 'Data Scientist',
                 'Data Science Manager': 'Data Scientist',
                 'Head of Data': 'Data Scientist',
                 'Applied Data Scientist': 'Data Scientist',
                 'Marketing Data Analyst': 'Data Analyst',
                 'Financial Data Analyst': 'Data Analyst',
                 'Machine Learning Developer': 'Data Scientist',
                 'Applied Machine Learning Scientist': 'Data Scientist',
                 'Data Analytics Manager': 'Data Analyst',
                 'Head of Data Science': 'Data Scientist',
                 'Data Specialist': 'Data Scientist',
                 'Data Architect': 'Data Engineer',
                 'Principal Data Analyst': 'Data Analyst',
                 'Staff Data Scientist': 'Data Scientist',
                 'Big Data Architect': 'Data Engineer',
                 'Analytics Engineer': 'Data Engineer',
                 'ETL Developer': 'Data Engineer',
                 'Head of Machine Learning': 'Data Engineer',
                 'NLP Engineer': 'Data Engineer',
                 'Lead Machine Learning Engineer': 'Data Engineer',
                 'Data Analytics Lead': 'Data Analyst',
                 'Big Data Engineer': 'Data Engineer',
                 'Machine Learning Engineer': 'Data Engineer',
                 'Lead Data Engineer': 'Data Engineer',
                 'Machine Learning Infrastructure Engineer': 'Data Engineer',
                 'ML Engineer': 'Data Engineer',
                 'Computer Vision Engineer': 'Data Engineer',
                 'Data Analytics Engineer': 'Data Engineer',
                 'Cloud Data Engineer': 'Data Engineer',
                 'Computer Vision Software Engineer': 'Data Engineer',
                 'Director of Data Engineering': 'Data Engineer',
                 'Data Science Engineer': 'Data Engineer',
                 'Principal Data Engineer': 'Data Engineer',
                 '3D Computer Vision Researcher': 'Data Scientist',
                 'Data Engineering Manager': 'Data Engineer',
                 'Finance Data Analyst': 'Data Analyst'})

In [86]:
df.value_counts("job_title",ascending=False)

job_title
Data Engineer     245
Data Scientist    235
Data Analyst      127
dtype: int64

In [87]:
pd.pivot_table(df, index=['work_year', 'experience_level', 'job_title'],aggfunc={'salary_in_usd':np.mean})

salary_in_usd
work_year experience_level job_title                    
2020      EN               Data Analyst     55814.400000
                           Data Engineer    95066.166667
                           Data Scientist   47055.888889
          EX               Data Engineer    79833.000000
                           Data Scientist  325000.000000
          MI               Data Analyst     60728.875000
                           Data Engineer    81275.500000
                           Data Scientist  103701.142857
          SE               Data Engineer    97011.000000
                           Data Scientist  177470.000000
2021      EN               Data Analyst     52536.666667
                           Data Engineer    49864.266667
                           Data Scientist   67693.652174
          EX               Data Analyst    150000.000000
                           Data Engineer   600000.000000
                           Data Scientist  190142.222222
          MI               Data Analyst    108398.818182
                           Data Engineer    82978.075000
                           Data Scientist   81605.076923
          SE               Data Analyst    116924.272727
                           Data Engineer   133248.193548
                           Data Scientist  122899.111111
2022      EN               Data Analyst     73000.000000
                           Data Engineer    64134.181818
                           Data Scientist   64202.285714
          EX               Data Analyst    120000.000000
                           Data Engineer   186153.250000
                           Data Scientist  196284.666667
          MI               Data Analyst     68970.590909
                           Data Engineer    86764.685714
                           Data Scientist  110133.323529
          SE               Data Analyst    118966.272727
                           Data Engineer   145900.833333
                           Data Scientist  161399.433333

In [88]:
pd.pivot_table(df, index=['work_year', 'experience_level', 'company_location', 'employment_type', 'job_title'], aggfunc={'salary_in_usd':np.mean})

salary_in_usd
work_year experience_level company_location employment_type job_title                    
2020      EN               DE               FT              Data Engineer    54742.000000
                                                            Data Scientist   55997.000000
                                            PT              Data Engineer    15966.000000
                           DK               FT              Data Scientist   45896.000000
                           FR               FT              Data Scientist   45618.500000
...                                                                                   ...
2022      SE               MX               FT              Data Engineer    60000.000000
                           NL               FT              Data Engineer    62651.000000
                           US               FT              Data Analyst    120795.000000
                                                            Data Engineer   154208.441176
                                                            Data Scientist  165937.872727

[203 rows x 1 columns]

Categorical features which are not ordinal:
- employee_residence
- company_location
- job_title
- employment_type

Categorical features which are ordinal:
- experience_level

In [89]:
df.experience_level.unique()

array(['MI', 'SE', 'EN', 'EX'], dtype=object)

EN: (Entry-level) / MI: (Mid-level) / SE: (Senior-level) / EX: (Expert-level) / Director

In [90]:
df.employment_type.unique()

array(['FT', 'CT', 'PT', 'FL'], dtype=object)

PT (Part-time); FT (Full-time); CT (Contact); FL (Freelance)

In [91]:
df.job_title.unique()

array(['Data Scientist', 'Data Engineer', 'Data Analyst'], dtype=object)

In [92]:
df.company_location.unique()

array(['DE', 'JP', 'GB', 'HN', 'US', 'HU', 'NZ', 'FR', 'IN', 'PK', 'CN',
       'GR', 'AE', 'NL', 'MX', 'CA', 'AT', 'NG', 'ES', 'PT', 'DK', 'IT',
       'HR', 'LU', 'PL', 'SG', 'RO', 'IQ', 'BR', 'BE', 'UA', 'IL', 'RU',
       'MT', 'CL', 'IR', 'CO', 'MD', 'KE', 'SI', 'CH', 'VN', 'AS', 'TR',
       'CZ', 'DZ', 'EE', 'MY', 'AU', 'IE'], dtype=object)

## Tiền xử lý

In [93]:
def not_usa(x):
    if x!='US':
        return "not_USA"
    else:
        return "USA"
    
df['company_location'] = df['company_location'].apply(not_usa)

In [94]:
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float] and df[col].nunique() == 2]
binary_cols

['company_location']

In [95]:
ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]
ohe_cols 

['work_year',
 'experience_level',
 'employment_type',
 'job_title',
 'remote_ratio',
 'company_size']

In [96]:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [99]:
# for col in binary_cols:
#     label_encoder(df, col)
# df = one_hot_encoder(df, ohe_cols)
df.head()

,salary_in_usd,employee_residence,company_location,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_Data Engineer,job_title_Data Scientist,remote_ratio_50,remote_ratio_100,company_size_M,company_size_S
0,79833,DE,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0
1,260000,JP,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1
2,109024,GB,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0
3,20000,HN,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1
4,150000,US,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0


In [100]:
X = df.drop(['salary_in_usd', 'employee_residence'], axis=1) # independent variables
y = df[["salary_in_usd"]] # dependent variable

In [101]:
X.head()

,company_location,work_year_2021,work_year_2022,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_Data Engineer,job_title_Data Scientist,remote_ratio_50,remote_ratio_100,company_size_M,company_size_S
0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0
1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1
2,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0
3,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0


In [42]:
y.head()

,salary_in_usd
0,79833
1,260000
2,109024
3,20000
4,150000


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

Pipeline để gom mô hình chuẩn hóa

In [44]:
pipe = Pipeline([('scaler', StandardScaler()), ('linear', LinearRegression())])

đánh giá

In [45]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('linear', LinearRegression())])

In [46]:
y_pred_test = pipe.predict(X_test)

In [47]:
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_test)))

RMSE: 42434.3595749739


In [48]:
print('MSE:', mean_squared_error(y_test, y_pred_test))

MSE: 1800674872.5381794


In [49]:
print('MAE:', mean_absolute_error(y_test, y_pred_test))

MAE: 32050.33490330401


In [50]:
print('Model score:', pipe.score(X_test, y_test))

Model score: 0.5540396248866011


In [ ]:
PCA(n_components = 8)

## Model selection

In [103]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

In [104]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(StandardScaler(), PolynomialFeatures(degree), LinearRegression(**kwargs))

In [105]:
param_grid = {'polynomialfeatures__degree': [3,4,5], 'linearregression__fit_intercept': [True, False], 'linearregression__normalize': [True, False]}

In [106]:
poly_grid = GridSearchCV(PolynomialRegression(), param_grid, 
                         cv=10, 
                         scoring='neg_mean_squared_error', 
                         verbose=3) 

In [107]:
poly_grid.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV 1/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-761566726792855730738681953166295040.000 total time=   0.0s
[CV 2/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-55503112039050594157847252259831808.000 total time=   0.0s


C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

[CV 3/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-4147193390455515320852397098373480448.000 total time=   0.0s
[CV 4/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-727231968717006014848607380890976256.000 total time=   0.0s
[CV 5/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-1947766099269248874365268472056774656.000 total time=   0.0s
[CV 6/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-27661111908553474580957519799123968.000 total time=   0.0s


C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

[CV 7/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-637142121010930676457158121944514560.000 total time=   0.0s
[CV 8/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-10065220116142263958083937312306102272.000 total time=   0.0s
[CV 9/10] END linearregression__fit_intercept=True, linearregression__normalize=True, polynomialfeatures__degree=3;, score=-30457307043667315566282682587414528.000 total time=   0.0s


...

[CV 9/10] END linearregression__fit_intercept=False, linearregression__normalize=False, polynomialfeatures__degree=5;, score=-172207264259150373191680000.000 total time=   0.6s


C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(


[CV 10/10] END linearregression__fit_intercept=False, linearregression__normalize=False, polynomialfeatures__degree=5;, score=-877405584761187924443136.000 total time=   0.4s


C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__fit_intercept': [True, False],
                         'linearregression__normalize': [True, False],
                         'polynomialfeatures__degree': [3, 4, 5]},
             scoring='neg_mean_squared_error', verbose=3)

In [108]:
print('Best Score: %s' % poly_grid.best_score_)
print('Best Hyperparameters: %s' % poly_grid.best_params_)

Best Score: -2.116119328259075e+26
Best Hyperparameters: {'linearregression__fit_intercept': False, 'linearregression__normalize': True, 'polynomialfeatures__degree': 5}


In [57]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDRegressor   

In [58]:
clf = SGDRegressor()

In [59]:
penalty = ['l1', 'l2']
loss = ['huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
param_distribs = dict(penalty=penalty, loss=loss)
rnd_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=10, cv=5, scoring='accuracy', random_state=42)
rnd_search.fit(X_train, y_train)
rnd_search.best_params_

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tran

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 93, in _check_targets
    raise 

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metric

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\tranq\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 93, in _check_targets
    raise 

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'penalty': 'l1', 'loss': 'huber'}

In [60]:
clf = SGDRegressor(penalty='l1', loss='huber')

In [61]:
clf.fit(X_train, y_train)

C:\Users\tranq\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tranq\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(loss='huber', penalty='l1')

In [62]:
y_pred = clf.predict(X_test)

In [63]:
from sklearn.metrics import classification_report

In [64]:
print(mean_absolute_error(y_test, y_pred))

111306.06924232232


In [109]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

X, y = fetch_california_housing(as_frame=True, return_X_y=True)

In [110]:
X = X[:10000]
y = y[:10000]

In [112]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
9995,4.0775,10.0,6.140900,1.025440,1275.0,2.495108,39.14,-121.03
9996,4.0848,8.0,6.350394,1.091864,1977.0,2.594488,39.13,-121.07
9997,3.6333,7.0,7.243455,1.107330,1143.0,2.992147,39.11,-121.05
9998,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,39.08,-121.04
